[![imagenes](imagenes/pythonista.png)](https://pythonista.mx)

# Flask a detalle.

## Componentes básicos de Flask.

Flask es conocido como un "microframework". Es decir, que a diferencia de proyectos como Django que viene "con las pilas incluidas", Flask sólamente contiene funcionalidades básicas tales como:

* Un servidor de aplicaciones basado en la biblioteca [Werkzeug](http://werkzeug.pocoo.org/).
* Una herramienta de depuración.
* Soporte de Plantillas por medio de [Jinja 2](http://jinja.pocoo.org).
* Soporte para pruebas unitarias.
* Soporte para cookies seguras.
* Soporte para desarrollo de instrucciones por medio de la interfaz de línea de comandos (CLI) mediante [Click](http://click.pocoo.org).


## Las extensiones de Flask.

Lo anterior no significa que Flask sea una herramienta limitada, sino que es altamente modular y el desarrollador puede seleccionar exclusivamente las extensiones y herramientas que mejor le plazcan.

Flask cuenta  con un completo catáĺogo de extensiones disponibles en la siguiente liga: http://flask.pocoo.org/extensions/

## La clase _flask.Flask_.

La clase _flask.Flask_ es el componente principal del framework. Los objetos instanciados a partir de esta clase realizarán todas las funciones del servidor de aplicaciones.

El único parámetro requerido obligatoriamente al instanciar un objeto de tipo _Flask_ es el nombre de la aplicación, el cual de principio corresponde al objeto asignado al nombre del entrono global *\_\_name\_\_*.

Solamente si se piensa en utilizar un objeto de tipo _Flask_ dentro de un paquete, el nombre deber de ser cambiado por el nombre del paquete.

**Sintaxis:**

``` python
<nombre> = flask.Flask(<nombre de la aplicación>)
```

**Ejemplo:**

En el script [*src/servidor.py*](src/servidor.py) se le asignó el nombre _app_ al objeto de tipo _Flask_.

```python
from flask import Flask

app = Flask(__name__)
```

## Funciones de vista y URL.
Cuando un cliente realiza una petición al servidor, ésta debe de corresponder a un URL válido. Es decir, que exista una ruta capaz de gestionar la petición.

Flask puede gestionar las peticiones a una URL válida mediante las "funciones de vista" (view functions). 

Las funciones de vista corresponden a una función que es definida aplicando un decorador que invoca a al método _route()_ del objeto de tipo _Flask_ la ruta corresondiente es ingresada como un parámetro en el método _route()_:

```
@<objeto de tipo Flask>.route('<ruta>', methods=[<lista de métodos>], <parámetros>)
def <nombre>(<parámetros>):
    ...
    ...
```

Las funciones de vista aceptan por defecto sólo el método GET. Para aceptar otros métodos es necesario ingresar el nombre del método con mayúsuclas como cadena de texto dentro del objeto tipo _list_ del parámetro _methods_.


** Ejemplo:**

En el código del script [*src/servidor.py*](src/servidor.py) se definió sólo una función de vista para la ruta raíz ("_/_") llamada _inicio()_.

```python
@app.route('/')
def inicio():
    return('<p>Hola, Mundo.</p>')
```
En el ejemplo anterior, la ruta es fija. Sin embargo, es posible que Flask pueda gestionar rutas dinámicamente utilizando "argumentos" cuyos nombres están encerradas entre los signos "_&lt;_" y "_&gt;_" con la siguiente sintaxis en el decorador del método _route()_:

El valor asignado a la variable de será tomado como parte de la ruta y el nombre será añadido al espacio de nombres del ámbito global.
```
@<objeto de tipo Flask>.route('<ruta><<parámetros>>')
```
**Ejemplo:**

El código de la siguiente celda define 2 funciones:
* Una función de vista ligada a una ruta estática.
* Una función de vista que creará una ruta dinámica correspondiente al texto que se ingrese después de la raíz de la URL, el nombre de la variable será añadido al espacio de nombres.

Al ejecutar el código que se incluye a continuación, se iniciará el servidor de Flask y escuchará las peticiones de todas las direcciones IP en el puerto 5000.

Para acceder a la función de vista _inicio()_, dirija su navegador a http://localhost:5000

Para acceder a la función de vista *ruta_dinamica()* use la siguiente sintaxis:
```
localhost:5000/<texto>
```
Por ejemplo, puede usar la dirección http://localhost:5000/Juan

**Advertencia:** Una vez ejecutada la siguiente celda, es necesario interrumpir el kernel de Jupyter para poder ejecutar el resto de las celdas de la notebook.

In [ ]:
from flask import Flask
app =Flask(__name__)


@app.route('/')
def inicio():
    return('<p>Hola, Mundo.</p>')
    
    
@app.route('/<usuario>')
def ruta_dinamica(usuario):
    return('<p>Hola, {}.</p>'.format(usuario))

#Si no se define el parámetro host, flask sólo será visible desde localhost
# app.run(host='localhost')
app.run(host="0.0.0.0", port=5000)

El script [*src/rutas.py*](src/rutas.py) contiene el código de la celda previa.

## Ejecución del servidor.

El método _run()_ del objeto de tipo _Flask_ es el encargado de levantar el servidor web. Es posible ingresar algunos parámetros iniciales tales como:

### El parámetro _host_.

Con este parámetro se define las direcciones IP que puede escuchar. Por defecto, sólo atiende a las peticiones de _localhost_, pero si se quisiera que el servidor escuchara a las peticiones porvenientes de cualquier IP, se ingresa la cadena de caracteres _"0.0.0.0"_.

### El parámetro _port_.

Con este parámetro se define el número del puerto que el servidor escucha. Por defecto escucha al puerto _5000_.

### El parámetro _debug_.

Este parámetro permite a Flask entrar en modo de depuración y su valor por defecto es _False_.

**Ejemplo:**

```
app.run(host="0.0.0.0", port=5000)
```

## Contextos de Flask.

Flask es un servidor con la capacidad de permitir a una aplicación ejecutarse en varios hilos (threads) y a su vez, cada aplicación puede darle servicio a diversos usuarios concurrentes, por lo que la gestión de ámbitos puede complicarse rápidamente.

Los desarrolladores de Flask definieron tres estados o "contextos" en los que puede encontrarse una aplicación.

* __El contexto de aplicación__ (application context) corresponde al estado cuando el objeto de tipo _Flask_ está en ejecución, pero no ha recibido una petición de algún cliente.
* __El contexto de petición__ (request context)corresponde al estado del objeto de tipo _Flask_ cuando un cliente realiza una petición. Este genera una especie de entorno global para cada petición que no interfiere con otras peticiones que se realizan simultaneamente.
* __Contexto de transición__ corresponde a un estado intermedio que puede ser aprovechado por medio de la ejecucion e comandos mediante la shell de Flask. A diferencia de los otros dos contextos, éste no tiene un nombre particular.

Durante la ejecución de un objeto de tipo _Flask_, éste puede cambiar de un contexto a otro (a esto se le conoce como _"push"_). Dependiendo de el contexto en el que se encuentre el objeto, éste podrá tener acceso a ciertos objetos en su ámbito global.

### Objetos en el contexto de aplicación.
* __El objeto *current_app*__ se refiere a la aplicación en ejecución.
* __El objeto *g*__ es un objeto capaz de guardar información temporal mientras se atiende a una petición.

### Objetos en el contexto de petición.
* __El objeto *request*__ contiene la petición hecha por el cliente.
* __El objeto *session*__ es un objeto de tipo _dict_ que puede ser utilizado para guardar información de un mismo cliente entre una petición y otra.

La razón de estos cambios de contexto es para proteger al objeto de tipo _Flask_ de código malicioso que pudiera ser inyectado desde una petición y que a su vez, cada petición cuente con un entorno aislado de las otras.


## El comando _flask_ y el shell de Flask.

A partir de la versión 0.11 de Flask se integró el comando _flask_, el cual puede ser ejecutado desde una terminal. Dicho comando permite:

* Ejecutar una aplicación de Flask mediante el comando _flask run_.
* Abrir un shell de Python que interactúa con el contexto de aplicación mediante el comando _flask shell_.

### Especificación de la aplicación de Flask.

Para poder utilizar el comando _flask_, es necesario declarar la variable de entorno _FLASK_APP_ con la ruta del script que contiene elcódigo de la aplicación de Flask.

** Sintaxis en Linux y MacOS X:**
``` 
export FLASK_APP = <ruta del script>
```
** Sintaxis en Windows:**
``` 
set FLASK_APP = <ruta del script>
```

### Modo de depuración.
Es posible habilitar el modo de depuración asignándole el valor de _1_ a la variable de entorno *FLASK_DEBUG*.

** Sintaxis en Linux y MacOS X:**
``` 
export FLASK_DEBUG=1
```
** Sintaxis en Windows:**
``` 
set FLASK_DEBUG=1
```

### El comando _flask run_.

Este comando ejecuta el contenido de un script que incluye un objeto de tipo _Flask_. Y en este caso, no es necesario incluir un método *run()* del objeto de tipo Flask en el código para que éste levante el servidor.

La designación de las direcciones IP autorizadas se definen con la opción *-h* y a continuación la cadena de caracteres con el segmento. 

La designación del puerto que utilizará la aplicación se definen con la opción *-p* y a continuación el número de puerto. 

**Sintaxis:**
```
flask run -h '<segmento>' -p <numero de puerto>
```

** Ejemplo:**

El script [*src/rutas-shell.py*](src/rutas-shell.py) contiene el siguiente código:

``` python
#! /usr/bin/python3

from flask import Flask
app =Flask(__name__)


@app.route('/')
def inicio():
    return('<p>Hola, Mundo.</p>')
    
    
@app.route('/<usuario>')
def ruta_dinamica(usuario):
    return('<p>Hola, {}.</p>'.format(usuario))
   
```

In [ ]:
%pwd

Asegúrese que la ruta de corresponda con la ruta en la que se encuentra el script _[src/rutas-shell.py](src/rutas-shell.py)_

**Advertencia:** Una vez ejecutada la siguiente celda, es necesario interrumpir el kernel de Jupyter para poder ejecutar el resto de las celdas de la notebook.

In [ ]:
%%bash
export FLASK_APP=/opt/pythonista/py201/src/rutas-shell.py
flask run -h '0.0.0.0'

Esta celda levantará un servidor que puede ser accedido desde http://localhost:5000

### El comando _flask shell_.

El comando _flask shell_ abre un entorno interactivo de Python que incluye al ámbito global del contexto de aplicación del objeto de tipo _Flask_ que se creó en el script inidcado mediante la variable de entorno *FLASK_APP*.

** NOTA:** El shell de Flask no funciona adecuadamente desde una notebook de Jupyter, por lo que es necesario ejecutarlo desde una terminal.

**Ejemplo:**

Se habilitará un shell de Flask desde el directorio en el que se encuentra el script _rutas-shell,py_

Desde una terminal de Linux se ejecutará los siguientes comandos:

``` bash
$ export FLASK_APP=rutas-shell.py
$ flask shell
```
Esto abrirá un entorno interactivo de Python que contiene el ámbito global correspondiente al contexto de aplicación del objeto de tipo Flask creado en el script _rutas-shell,py_.

``` python
Python 3.4.5 (default, May 29 2017, 15:17:55)
[GCC 4.8.5 20150623 (Red Hat 4.8.5-11)] on linux
App: rutas-shell
Instance: /home/oi/py201_Flask_y_REST_API/instance
>>> dir()
['__builtins__', 'app', 'g']
>>>
```

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2018.</p>